<a href="https://colab.research.google.com/github/viincci/viincci/blob/main/RasaTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U ipython

In [ ]:
pip install pandas rasa


In [4]:
import os
import pandas as pd
from rasa.shared.nlu.training_data.message import Message
from rasa.shared.nlu.training_data import TrainingData
# from rasa.nlu.training_data import Message
# from rasa.nlu.training_data import TrainingData

def add_parquet_data_to_rasa_data(parquet_file_path, rasa_data_folder):
    # Read data from Parquet file
    pqfl = pd.read_parquet(parquet_file_path).to_dict()

    # Create a dictionary to store the intents and responses
    ntnt_n_rspns = {}

    # Group questions and corresponding responses
    for k, v in pqfl['question'].items():
        ntnt_n_rspns[v] = ntnt_n_rspns.get(v, []) + [str(k)]

    for k, v in ntnt_n_rspns.items():
        ntnt_n_rspns[k] = ''.join(pqfl['answer'][int(itm)] for itm in v)

    # Create Rasa TrainingData object
    training_data = TrainingData()

    # Add intents and responses to Rasa TrainingData
    for intent, response in ntnt_n_rspns.items():
        message = Message.build(text=intent, intent=intent, response=response)
        training_data.training_examples.append(message)

    # Write TrainingData to files in Rasa data folder
    nlu_data_file = os.path.join(rasa_data_folder, 'nlu.yml')
    training_data.persist_nlu(nlu_data_file)

    # Optionally, you can also create and save stories if you have corresponding story data
    # stories_data_file = os.path.join(rasa_data_folder, 'stories.yml')
    # training_data.persist_stories(stories_data_file)

# Example usage
parquet_file_path = "train-1mb.parquet"
rasa_data_folder = "../path/to/your/rasa/data/folder"

add_parquet_data_to_rasa_data(parquet_file_path, rasa_data_folder)
